In [23]:
df = pd.read_csv('C:/Users/oyeye/EVProject/evdata.csv')

In [25]:
"""
Dataset Description:

This dataset includes 18 columns related to electric vehicles (EVs), renewable energy, and market forecasting. 
Each column is described below with its purpose and data type.

Columns:
    1. vehicle_id (string): Unique identifier for each vehicle.
    2. ev_sales_current_year (int32): The number of EVs sold in the current year.
    3. ev_sales_previous_year (int32): The number of EVs sold in the previous year.
    4. renewable_energy_share (float64): The percentage of energy derived from renewable sources (e.g., solar, wind) in decimal form.
    5. battery_capacity (float64): The battery capacity of the EV in kilowatt-hours (kWh).
    6. charger_power (int32): The charging power of the EV's charger in kilowatts (kW).
    7. annual_distance (float64): The total annual distance traveled by the EV in miles.
    8. maintenance_cost_ev (float64): The annual maintenance cost of the EV in dollars.
    9. maintenance_cost_ffv (float64): The annual maintenance cost of fossil fuel vehicles (FFVs) in dollars.
    10. emissions_reduction (float64): The total reduction in CO2 emissions by using the EV compared to an FFV, measured in metric tons.
    11. base_price (float64): The base price of the EV in dollars.
    12. recycling_cost (float64): The cost associated with recycling the EV's battery at the end of its life cycle in dollars.
    13. second_life_revenue (float64): Revenue generated by repurposing EV batteries for second-life applications in dollars.
    14. forecast_years (int32): The number of years into the future for which projections are made.
    15. charging_time_hours (float64): The time required to fully charge the EV battery in hours.
    16. cost_per_mile (float64): The cost of driving one mile in the EV in dollars.
    17. batteries_recycled (int32): The number of EV batteries recycled in a given year.
    18. profit (float64): The profit margin per EV, calculated as revenue minus total costs, in dollars.

Note:
This dataset can be used for analyzing trends in EV adoption, profitability, and renewable energy integration. It also supports market forecasting and environmental impact assessments.
"""

"\nDataset Description:\n\nThis dataset includes 18 columns related to electric vehicles (EVs), renewable energy, and market forecasting. \nEach column is described below with its purpose and data type.\n\nColumns:\n    1. vehicle_id (string): Unique identifier for each vehicle.\n    2. ev_sales_current_year (int32): The number of EVs sold in the current year.\n    3. ev_sales_previous_year (int32): The number of EVs sold in the previous year.\n    4. renewable_energy_share (float64): The percentage of energy derived from renewable sources (e.g., solar, wind) in decimal form.\n    5. battery_capacity (float64): The battery capacity of the EV in kilowatt-hours (kWh).\n    6. charger_power (int32): The charging power of the EV's charger in kilowatts (kW).\n    7. annual_distance (float64): The total annual distance traveled by the EV in miles.\n    8. maintenance_cost_ev (float64): The annual maintenance cost of the EV in dollars.\n    9. maintenance_cost_ffv (float64): The annual mainte

In [33]:
# EV Maintenance Cost Calculation

def adjusted_maintenance_cost(df):
    """    
    Calculate the adjusted maintenance cost for electric vehicles (EVs).

    This function computes an adjusted maintenance cost for each vehicle 
    in a DataFrame, factoring in sales growth. If there is a growth in 
    EV sales compared to the previous year, the adjusted cost incorporates 
    a scaled factor of fossil fuel vehicle (FFV) maintenance costs. If 
    there were no sales in the previous year, only the EV maintenance cost 
    is used.

    Parameters:
        df (pd.DataFrame): A pandas DataFrame containing the following columns:
            - vehicle_id (string): Unique identifier for each vehicle.
            - ev_sales_current_year (int): EV sales for the current year.
            - ev_sales_previous_year (int): EV sales for the previous year.
            - maintenance_cost_ev (float): Annual maintenance cost for the EV in dollars.
            - maintenance_cost_ffv (float): Annual maintenance cost for fossil fuel vehicles in dollars.

    Returns:
        pd.DataFrame: A DataFrame with the following columns:
            - vehicle_id (string): The unique vehicle identifier.
            - adj_maintenance_cost (float): The calculated adjusted maintenance cost.

    Calculation Logic:
        1. If EV sales in the previous year (`ev_sales_previous_year`) > 0:
           - Calculate the sales growth factor as:
             `sales_growth_factor = ev_sales_current_year / ev_sales_previous_year`
           - Adjusted maintenance cost as:
             `adj_maintenance_cost = maintenance_cost_ev + (maintenance_cost_ffv * sales_growth_factor)`
        2. If EV sales in the previous year = 0:
           - Use the EV maintenance cost only:
             `adj_maintenance_cost = maintenance_cost_ev`
    """
    
    def calc_adjusted_cost(row):
        if row['ev_sales_previous_year'] > 0:
            sales_growth_factor = row['ev_sales_current_year'] / row['ev_sales_previous_year']
            adj_maintenance_cost = row['maintenance_cost_ev'] + (row['maintenance_cost_ffv'] * sales_growth_factor)
        else:
            adj_maintenance_cost = row['maintenance_cost_ev']
            
        return adj_maintenance_cost
        
    df['adj_maintenance_cost'] = df.apply(calc_adjusted_cost, axis=1)
        
    return df[['vehicle_id', 'adj_maintenance_cost']]

In [28]:
# EV Recycling Revenue Optimization

def net_recycling_revenue(df):
    def calc_rev(row):
        
        # Ensure net revenue isn't negative
        if row['recycling_cost'] > row['second_life_revenue']:
            net_rev = 0
        
        # Taking into consideration where no batteries are recycled
        elif row['batteries_recycled'] == 0:
            net_rev = 0
        
        # Calculate net recycling revenue
        else:
            net_rev = row['batteries_recycled'] * (row['second_life_revenue'] - row['recycling_cost'])
            
        return net_rev
    
    df['net_recycling_revenue'] = df.apply(calc_rev, axis = 1)
    return df[['vehicle_id', 'net_recycling_revenue']]

In [68]:
# EV Demand and Supply
# Price Prediction Based on Market Dynamics
def price_prediction(df):
    
    def calc_pp(row):
        sales_growth = (row['ev_sales_current_year'] - row['ev_sales_previous_year']) / row['ev_sales_previous_year']
        
        # Cap renewable_energy_share and sales_growth to maximum of 50%
        if row['renewable_energy_share'] + sales_growth > 0.5:
            row['renewable_energy_share'] + sales_growth == 0.5
            
        
        adj_price = row['base_price'] * (1 + row['renewable_energy_share'] + sales_growth)
        
        # Ensure adjusted price not below base price
        return max(adj_price, row['base_price'])
        
    df['adj_price'] = df.apply(calc_pp, axis = 1)
    return df[['vehicle_id', 'adj_price']]

In [75]:
#########EV Net Cost Per Mile Optimization
def net_cost_per_mile(df):
    def calc_net_cost(row):
                
        #######Apply 15% discount to recycling_cost
        if row['batteries_recycled'] > 1000:
            row['recycling_cost'] = 0.85 * row['recycling_cost']
        
        #######Apply 10% surcharge to recycling_cost
        elif row['batteries_recycled'] < 500:
            row['recycling_cost'] = 1.1 * row['recycling_cost']
        
        ######Apply 10% bonus to second_life_revenue
        if row['renewable_energy_share'] > 0.5:
            row['second_life_revenue'] = 1.1 * row['second_life_revenue']
            
        
        ######Apply 20% penalty to cost_per_mile
        if row['annual_distance'] < 10000:
            estimated_cost_per_mile = 1.2 * ((row['maintenance_cost_ev'] + row['recycling_cost'] - row['second_life_revenue']) / row['annual_distance'])
            
        else:
             estimated_cost_per_mile = 1 * ((row['maintenance_cost_ev'] + row['recycling_cost'] - row['second_life_revenue']) / row['annual_distance'])
            
        
        return max(estimated_cost_per_mile, row['cost_per_mile'])
    
    df['est_cost_per_mile'] = df.apply(calc_net_cost, axis = 1)
    return df[['vehicle_id', 'est_cost_per_mile']]

In [91]:
########Forecasting EV Sales Growth
def forecast_sales(df):
    def calc_forecast(row):
        
        
        ######Cap Growth Rate at -10% to prevent over-penalization
        sales_growth_rate = (row['ev_sales_current_year'] - row['ev_sales_previous_year']) / row['ev_sales_previous_year']
        
        if row['ev_sales_current_year'] < row['ev_sales_previous_year']:
            sales_growth_rate = -0.1
            
        else:
            sales_growth_rate = sales_growth_rate
        
        
        ######Add 5% Growth Rate for High Renewable Energy Share
        if row['renewable_energy_share'] > 0.5:
            sales_growth_rate = 1.05 * sales_growth_rate
        
        else:
            sales_growth_rate = 1 * sales_growth_rate
            
        
        adjusted_growth_rate = sales_growth_rate * (1 + (row['renewable_energy_share'] / 100))
        
        
        if(adjusted_growth_rate < 0.01 and row['renewable_energy_share'] > 0.3):
            adjusted_growth_rate = max(adjusted_growth_rate, 0.02)
            
        projected_sales = row['ev_sales_current_year'] * (1 + adjusted_growth_rate)
        
        return min(projected_sales, 1.5 * row['ev_sales_current_year'])
    
    df['sales_forecast'] = df.apply(calc_forecast, axis = 1)
    
    return df[['vehicle_id', 'sales_forecast']]            

In [13]:
def lifetime_maintenance_cost_comparison(df):
    def cost_comparison(row):
        maintenance_cost_ev = row['maintenance_cost_ev']
        maintenance_cost_ffv = row['maintenance_cost_ffv']
        forecast_years = row['forecast_years']
        
        lifetime_maintenance_ev = 0
        lifetime_maintenance_ffv = 0
        
        for year in range(1, forecast_years + 1):
            if year > 10:
                maintenance_cost_ev = 1.05 * maintenance_cost_ev
                maintenance_cost_ffv = 1.05 * maintenance_cost_ffv
                
                maintenance_cost_ev = min(maintenance_cost_ev, 1.15 * maintenance_cost_ev)
                maintenance_cost_ffv = min(maintenance_cost_ffv, 1.15 * maintenance_cost_ffv)
                
            lifetime_maintenance_ev = lifetime_maintenance_ev + maintenance_cost_ev
            lifetime_maintenance_ffv = lifetime_maintenance_ffv + maintenance_cost_ffv
            
        cost_advantage = lifetime_maintenance_ffv - lifetime_maintenance_ev
        highlight_row = lifetime_maintenance_ev <= 0.7 * lifetime_maintenance_ffv
        
        return pd.Series({'lifetime_maintenance_ev' : lifetime_maintenance_ev,
                         'lifetime_maintenance_ffv' : lifetime_maintenance_ffv,
                         'cost_advantage' : cost_advantage,
                         'highlight_row' : highlight_row
                         })
    
    maintenance = df.apply(cost_comparison, axis = 1)
    df = pd.concat([df, maintenance], axis = 1)
    return df

In [41]:
def battery_recycling_profitability(df):
    def profitability(row):
        batteries_recycled = row['batteries_recycled']
        recycling_cost = row['recycling_cost']
        second_life_revenue = row['second_life_revenue']
        forecast_years = row['forecast_years']
        
        cum_profit = 0
        
        for year in range(1, forecast_years + 1):
            batteries_recycled = 1.02 * batteries_recycled
            annual_profit = batteries_recycled * (second_life_revenue - recycling_cost)
            cum_profit = cum_profit + annual_profit
            
            
        if recycling_cost > second_life_revenue:
            new_col = 'Unprofitable'
            cum_profit = 0
            
        else:
            new_col = 'Profitable'
            
        if cum_profit > 1000000:
            profit_indication = 'Profitable'
            
        else:
            profit_indication = 'UnProfitable'
        
        return pd.Series({'cum_profit' : cum_profit,
                          'new_col' : new_col,
                          'profit_indication' : profit_indication
                         })
    

    battery_profitability = df.apply(profitability, axis = 1)
    df = pd.concat([df, battery_profitability], axis = 1)
    return df

In [48]:
# EV Sales Growth Rate Analysis

def ev_sales_growth(df):
    def g_rate(row):
        ev_sales_current_year = row['ev_sales_current_year']
        ev_sales_previous_year = row['ev_sales_previous_year']
        benchmark = 15
        
        growth_rate = ((ev_sales_current_year - ev_sales_previous_year) / ev_sales_previous_year) * 100
        
        # Growth rate below assigned benchmark
        if growth_rate < benchmark:
            Flagged = 'Below Benchmark'
            
        else:
            Flagged = 'Not Below'
        
        # Negative Growth Check
        if ev_sales_current_year < ev_sales_previous_year:
            Growth_check = 'Declining Market'
        
        else:
            Growth_check = 'Stable Market'
            
        # Set Performance Categories
        if growth_rate >= benchmark + 5:
            Perf_category = 'Outstanding'
            
        elif benchmark < growth_rate < benchmark + 5:
            Perf_category = 'Good'
            
        elif growth_rate < benchmark:
            Perf_category = 'Underperforming'
            
        return pd.Series({'growth_rate' : growth_rate, 
                         'Flagged' : Flagged,
                         'Growth_check' : Growth_check,
                         'Perf_category' : Perf_category})
    
    ev_sales = df.apply(g_rate, axis = 1)
    df = pd.concat([df, ev_sales], axis = 1)
    
    return df

In [56]:
def annual_maintenance_cost_comparison(df):
    def maintenance_cost(row):
        maintenance_cost_ev = row['maintenance_cost_ev']
        maintenance_cost_ffv = row['maintenance_cost_ffv']
        annual_distance = row['annual_distance']
        
        cost_per_mile_ev = maintenance_cost_ev / annual_distance
        cost_per_mile_ffv = maintenance_cost_ffv / annual_distance
        
        savings = cost_per_mile_ffv - cost_per_mile_ev
        
        
        # Set Savings Threshold
        if savings < 0:
            flagged = 'Negative'
            
        else:
            flagged = 'Positive'
        
        # Categorize Savings
        if savings >= 0.05:
            cat_savings = 'High Savings'
            
        elif 0.01 < savings < 0.05:
            cat_savings = 'Moderate Savings'
            
        else:
            cat_savings = 'Low Savings'
            
            
        return pd.Series({'cost_per_mile_ev' : cost_per_mile_ev,
                          'cost_per_mile_ffv' : cost_per_mile_ffv,
                        'savings' : savings,
                         'flagged' : flagged,
                         'cat_savings' : cat_savings,
                         })
    
    maintenance = df.apply(maintenance_cost, axis = 1)
    df = pd.concat([df, maintenance], axis = 1)
    return df

In [90]:
def annual_maintenance_cost_comparison(df):
    def cost_comparison(row):
        maintenance_cost_ev = row['maintenance_cost_ev']
        maintenance_cost_ffv = row['maintenance_cost_ffv']
        forecast_years = row['forecast_years']
        
        cum_ev_maintenance_cost = []
        cum_ffv_maintenance_cost = []
        
        for year in range(1, forecast_years + 1):
            
            # Apply 5% growth factor for maintenance costs beyond 10 years
            if year > 10:
                maintenance_cost_ev = maintenance_cost_ev * 1.05
                maintenance_cost_ffv = maintenance_cost_ffv * 1.05
           
            # Do not apply growth factor for less than 10 years
            else:
                maintenance_cost_ev = maintenance_cost_ev * 1
                maintenance_cost_ffv = maintenance_cost_ffv * 1
            
            
            # Cap total growth for both EVs and FFVs to 15% of the initial cost
            maintenance_cost_ev = min(maintenance_cost_ev, 1.15 * maintenance_cost_ev)
            maintenance_cost_ffv = min(maintenance_cost_ffv, 1.15 * maintenance_cost_ffv)
            
            cum_ev_maintenance_cost.append(maintenance_cost_ev)
            cum_ffv_maintenance_cost.append(maintenance_cost_ffv)
        
        
        diff_maintenance_cost_ev = sum(cum_ev_maintenance_cost) - maintenance_cost_ev
        diff_maintenance_cost_ffv = sum(cum_ffv_maintenance_cost) - maintenance_cost_ffv
        
        cost_diff = sum(cum_ffv_maintenance_cost) - sum(cum_ev_maintenance_cost)
        
        return pd.Series({'cum_ev_maintenance_cost' : sum(cum_ev_maintenance_cost),
                         'cum_ffv_maintenance_cost' : sum(cum_ffv_maintenance_cost),
                         'diff_maintenance_cost_ev' : diff_maintenance_cost_ev,
                         'diff_maintenance_cost_ffv' : diff_maintenance_cost_ffv,
                         'cost_diff' : cost_diff})
    
    maintenance_comp = df.apply(cost_comparison, axis = 1)
    df = pd.concat([df, maintenance_comp], axis = 1)
    return df

In [94]:
# Battery Recycling Profitability Analysis
def battery_recycling_profitability(df):
    def profitability_analysis(row):
        recycling_cost = row['recycling_cost']
        second_life_revenue = row['second_life_revenue']
        batteries_recycled = row['batteries_recycled']
        forecast_years = row['forecast_years']
        
        adj_recycling_cost = []
        adj_second_life_revenue = []
        
        for year in range(1, forecast_years + 1):
            recycling_cost = recycling_cost * 1.02
            second_life_revenue = second_life_revenue * 1.03
            
            adj_recycling_cost.append(recycling_cost)
            adj_second_life_revenue.append(second_life_revenue)
        
        new_second_life_revenue = min(sum(adj_second_life_revenue), 1.3 * second_life_revenue)
            
        cum_net_recycling_rev = (new_second_life_revenue - sum(adj_recycling_cost)) * batteries_recycled
        
        average_net = cum_net_recycling_rev / forecast_years
        
        return pd.Series({'new_second_life_revenue' : new_second_life_revenue,
                         'cum_net_recycling_rev' : cum_net_recycling_rev,
                         'average_net' : average_net})
    
    recycling_profitability = df.apply(profitability_analysis, axis = 1)
    df = pd.concat([df, recycling_profitability], axis = 1)
    return df

In [101]:
# Charging Time Efficiency Analysis
def charging_time_efficiency_analysis(df, tolerance = 0.1):
    def charg_efficiency(row):
        charger_power = row['charger_power']
        battery_capacity = row['battery_capacity']
        charging_time_hours = row['charging_time_hours']
        
        # Expected Charging Time
        charging_time_expected = battery_capacity / charger_power
        
        # Effciency Rate
        efficiency_rate = (charging_time_expected / charging_time_hours) * 100
        
        # Efficiency Categorization
        if efficiency_rate >= 95:
            eff_cat = 'Highly Efficient'
            
        elif 85 <= efficiency_rate < 95:
            eff_cat = 'Moderately Efficient'
            
        else:
            eff_cat = 'Inefficient'
            
            
        # Tolerance Check
        if abs(charging_time_hours - charging_time_expected) > tolerance * charging_time_expected:
            tolerance_check = 'Flag'
            
        else:
            tolerance_check = 'Flag Not'
            
        return pd.Series({'charging_time_expected' : charging_time_expected,
                         'efficiency_rate' : efficiency_rate,
                         'eff_cat' : eff_cat,
                         'tolerance_check' : tolerance_check})
    
    charging_eff = df.apply(charg_efficiency, axis = 1)
    df = pd.concat([df, charging_eff], axis = 1)
    return df

In [112]:
# Cost Analysis
def cost_analysis(df):
    def calc_cost(row):
        base_price = row['base_price']
        maintenance_cost_ev = row['maintenance_cost_ev']
        maintenance_cost_ffv = row['maintenance_cost_ffv']
        recycling_cost = row['recycling_cost']
        
        total_cost = base_price + maintenance_cost_ev + maintenance_cost_ffv + recycling_cost
        
        if total_cost > 50000:
            cost_eval = 'Premium'
            
        elif total_cost < 30000:
            cost_eval = 'Budget'
            
        else:
            cost_eval = 'None'
            
        return pd.Series({'total_cost' : total_cost,
                         'cost_eval' : cost_eval})
    
    cost = df.apply(calc_cost, axis = 1)
    df = pd.concat([df, cost], axis = 1)
    return df

In [130]:
# Emissions Reduction Analysis
def emissions_reduction_analysis(df):
    def calc_emissions(row):
        emissions_reduction = row['emissions_reduction']
        annual_distance = row['annual_distance']
        
        emissions_reduction_tons = emissions_reduction * annual_distance
        
        if emissions_reduction_tons > 100:
            vehicle_cat = 'High Impact'
            
        elif emissions_reduction_tons < 10:
            vehicle_cat = 'Low Impact'
            
        else:
            vehicle_cat = 'No Impact'
            
        return pd.Series({'emissions_reduction_tons' : emissions_reduction_tons,
                         'vehicle_cat' : vehicle_cat})
    
    emissions = df.apply(calc_emissions, axis = 1)
    df = pd.concat([df, emissions], axis = 1)
    return df

In [134]:
# Battery Recycling Efficiency Analysis
def battery_recycling_eff(df):
    def calc_eff(row):
        batteries_recycled = row['batteries_recycled']
        recycling_cost = row['recycling_cost']
        second_life_revenue = row['second_life_revenue']
        
        net_rev_battery = (second_life_revenue / batteries_recycled) - (recycling_cost / batteries_recycled)
        
        if (recycling_cost / batteries_recycled) > (second_life_revenue / batteries_recycled):
            recycling_eff = 'Inefficient Recycling'
            
        else:
            recycling_eff = 'Efficient Recycling'
            
            
        if net_rev_battery > 100:
            rev_perf = 'High Revenue Recycling'
            
        elif net_rev_battery < 0:
            rev_perf = 'Net Loss Recycling'
            
        else:
            rev_perf = 'Moderate Revenue Recycling'
            
        return pd.Series({'net_rev_battery' : net_rev_battery,
                         'recycling_eff' : recycling_eff,
                         'rev_perf' : rev_perf})
    
    efficiency = df.apply(calc_eff, axis = 1)
    df = pd.concat([df, efficiency], axis = 1)
    return df

In [155]:
# Total Cost and Net Benefit Analysis
def net_benefit_cost(df):
    def calc_net_benefit(row):
        base_price = row['base_price']
        maintenance_cost_ev = row['maintenance_cost_ev']
        maintenance_cost_ffv = row['maintenance_cost_ffv']
        annual_distance = row['annual_distance']
        cost_per_mile = row['cost_per_mile']
        forecast_years = row['forecast_years']
        emissions_reduction = row['emissions_reduction']
        
        total_cost_ev = base_price
        total_cost_ffv = base_price * 1.2
        cum_net_benefit = 0
        cost_per_mile_ffv = 0.15
        
        # Total Cost for EVs over the forecast period
        for year in range(1, forecast_years + 1):
            if year > 10:
                maintenance_cost_ev = 1.05 * maintenance_cost_ev
                maintenance_cost_ev = min(maintenance_cost_ev_inc, 1.15 * row['maintenance_cost_ev'])
                
                maintenance_cost_ffv = 1.05 * maintenance_cost_ffv
                maintenance_cost_ffv = min(maintenance_cost_ffv, 1.15 * row['maintenance_cost_ffv'])
                
            total_cost_ev += maintenance_cost_ev + (annual_distance * cost_per_mile)
            total_cost_ffv += maintenance_cost_ffv + (annual_distance * cost_per_mile_ffv)
        
        # Apply 10% annual discount rate to account for diminishing returns in the value of emissions reductions
            net_benefit = total_cost_ffv - (total_cost_ev - 1.1 * emissions_reduction)
            cum_net_benefit += net_benefit
            
        
        if cum_net_benefit > 0.2 * total_cost_ffv:
            cost_advantage = 'High Benefit'
            
        elif cum_net_benefit < 0:
            cost_advantage = 'Not Cost Effective'
            
        else:
            cost_advantage = 'No Benefit'
            
            
        return pd.Series({'cost_advantage' : cost_advantage,
                         'cum_net_benefit' : cum_net_benefit})
    
    cost_net_benefit = df.apply(calc_net_benefit, axis = 1)
    df = pd.concat([df, cost_net_benefit], axis = 1)
    return df

In [157]:
# Renewable Energy Contribution
def renewable_energy_contribution(df):
    def calc_ren_cont(row):
        renewable_energy_share = row['renewable_energy_share']
        emissions_reduction = row['emissions_reduction']
        profit = row['profit']
        
        adj_emissions_red = emissions_reduction * renewable_energy_share
        cost_effectiveness_score = profit / adj_emissions_red
        
        if renewable_energy_share < 0.2:
            impact = 'Low Renewable Contribution'
            
        else:
            impact = 'High Renewable Contribution'
            
        if (adj_emissions_red > 0.9 * emissions_reduction and cost_effectiveness_score > 10):
            sustainability_class = 'Outstanding Sustainability'
            
        else:
            sustainability_class = 'Not Outstanding'
            
        if profit < 0:
            profit_class = 'Unsustainable Investment'
            
        else:
            profit_class = 'Sustainable Investment'
            
        if 5 < cost_effectiveness_score < 10:
            perf_class = 'Balanced Performance'
            
        else:
            perf_class = 'Unbalanced Performance'
            
        return pd.Series({'adj_emissions_red' : adj_emissions_red,
                         'cost_effectiveness_score' : cost_effectiveness_score,
                         'impact' : impact,
                         'sustainability_class' : sustainability_class,
                         'profit_class' : profit_class,
                         'perf_class' : perf_class})
    
    renewable_contribution = df.apply(calc_ren_cont, axis = 1)
    df = pd.concat([df, renewable_contribution], axis = 1)
    return df

In [169]:
# Sustainability Index
# Overall sustainability index of each EV model, incorporating environmental, economic, and energy factors
def sustainability_index(df):
    def calc_sustainability(row):
        renewable_energy_share = row['renewable_energy_share']
        emissions_reduction = row['emissions_reduction']
        profit = row['profit']
        charging_time_hours = row['charging_time_hours']
        annual_distance = row['annual_distance']
        battery_capacity = row['battery_capacity']
        
        # Efficiency Score
        # Combines charging time and battery capacity
        efficiency_score = battery_capacity / charging_time_hours
        
        # Economic-Environmental Ratio (EER)
        # Balances profit and emissions reduction
        eer = profit / emissions_reduction
        
        # Sustainability Index
        si = (efficiency_score * renewable_energy_share) + eer
        
        # Efficiency Warning
        if efficiency_score < 10:
            eff_class = 'Low Efficiency'
            
        else:
            eff_class = 'High Efficiency'
        
        # Economic-Environmental Ratio Impact
        if (eer > 500 and emissions_reduction < 5):
            profit_class = 'Profit-Focused Model'
            
        else:
            profit_class = 'No Profit'
        
        # Sustainability Classes
        if si > 50:
            sus_class = 'Highly Sustainable'
            
        elif 25 < si < 50:
            sus_class = 'Balanced Contribution'
            
        else:
            sus_class = 'Low Sustainability'
            
        return pd.Series({'efficiency_score' : efficiency_score,
                         'eer' : eer,
                         'si' : si,
                         'eff_class' : eff_class,
                         'profit_class' : profit_class,
                         'sus_class' : sus_class})
    
    sustainability = df.apply(calc_sustainability, axis = 1)
    df = pd.concat([df, sustainability], axis = 1)
    return df        

In [176]:
# Battery Recycling Optimization
# Optimization of EV batteries recycling to minimize costs and maximize revenue while maintaining environmental benefits

def battery_recycling_optimization(df):
    def optimization(row):
        recycling_cost = row['recycling_cost']
        batteries_recycled = row['batteries_recycled']
        second_life_revenue = row['second_life_revenue']
        emissions_reduction = row['emissions_reduction']
        forecast_years = row['forecast_years']
        
        
        # Net Revenue per Battery
        if batteries_recycled == 0: # Prevent 0 division
            net_revenue_per_battery = 0
        
        else:
            net_revenue_per_battery = (second_life_revenue / batteries_recycled) - (recycling_cost / batteries_recycled)
        
        # Total Net Revenue
        total_net_rev = second_life_revenue - recycling_cost
        
        # Environmental Impact Ratio (EIR)
        if recycling_cost == 0:  # Prevent 0 division
            eir = 0
            
        else:
            eir = emissions_reduction / recycling_cost
        
        # Optimization Score
        # Combines financial and environmental metrics
        optimization_score = (net_revenue_per_battery * 0.7) + (eir * 0.3)
        
        # Profit Class
        if net_revenue_per_battery < 0:
            profit_class = 'Not Profitable'
        else:
            profit_class = 'Profitable'
        
        # Environmental Impact Warning
        if eir < 0.1:
            impact = 'Low Environmental Impact'
        else:
            impact = 'High Environmental Impact'
        
        # Optimization Classes
        if optimization_score > 10:
            optimization_class = 'Highly Optimized'
            
        elif 5 < optimization_score < 10:
            optimization_class = 'Moderately Optimized'
            
        else:
            optimization_class = 'Needs Improvement'
            
            
        return pd.Series({'net_revenue_per_battery' : net_revenue_per_battery,
                          'total_net_rev' : total_net_rev,
                          'eir' : eir,
                          'optimization_score' : optimization_score,
                          'profit_class' : profit_class,
                          'impact' : impact,
                          'optimization_class' : optimization_class
                          })
    
    optimize = df.apply(optimization, axis = 1)
    df = pd.concat([df, optimize], axis = 1)
    return df

In [203]:
# Forecasted Revenue from Recycled Batteries
# Forecast revenue from recycling batteries over the years while accounting for improvements in efficiency
def forecasted_revenue(df):
    def calc_forecast(row):
        second_life_revenue = row['second_life_revenue']
        batteries_recycled = row['batteries_recycled']
        forecast_years = row['forecast_years']
        recycling_cost = row['recycling_cost']
        efficiency_improvement_rate = 0.03
        market_fluctuation_rate = 0.05
        incentives_per_battery = 50
        penalty_rate = 25
        efficiency_threshold = 0.8
        cumulative_profit = 0
        
        for year in range(1, forecast_years + 1):
            market_fluctuation = np.random.uniform(market_fluctuation_rate, -market_fluctuation_rate)
            
            # Adjusted Revenue
            adjusted_revenue = second_life_revenue * (1 + market_fluctuation)
            
            # Adjusted Cost
            adjusted_cost = 0.97 * recycling_cost * (1 - efficiency_improvement_rate * year)
            
            # Total Profit
            cumulative_profit += (adjusted_revenue - adjusted_cost) * batteries_recycled + incentives_per_battery
            
            # Check efficiency threshold for penalties
            efficiency = 0.75 + (efficiency_improvement_rate * year)
            penalty = penalty_rate * max(0, (efficiency_threshold - efficiency)) * batteries_recycled
            
            if (year > 5 and cumulative_profit < 0):
                classification = 'Non-Viable'
                
            elif cumulative_profit < 0:
                classification = 'Nothing' 
                
            else:
                classification = 'Viable' 
                
        return pd.Series({'cumulative_profit' : cumulative_profit,
                         'classification' : classification})
    
    results = df.apply(calc_forecast, axis = 1)
    df = pd.concat([df, results], axis = 1)
    return df